In [ ]:
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/Users/Linshaoqian/Downloads/ffmpeg"

import moviepy.editor as mp
# 显式地为moviepy设置ffmpeg路径

from pydub import AudioSegment

from google.cloud import speech

# 显式地设置 Google Application Credentials 环境变量
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcpai.json"

In [ ]:
# 将视频文件转换为音频
video = mp.VideoFileClip("周處除三害-77.mp4")
audio = video.audio
audio.write_audiofile("converted_audio-77.wav")

In [ ]:
# 读取音频文件
audio = AudioSegment.from_file("converted_audio-77.wav")

# 将立体声音频转换为单声道
mono_audio = audio.set_channels(1)

# 保存转换后的音频
mono_audio.export("77.wav", format="wav")

In [36]:
# 使用Google Speech-to-Text进行语音识别
client = speech.SpeechClient()

# 使用GCS上的音频文件
gcs_uri = "gs://movie77/77.wav"
audio = speech.RecognitionAudio(uri=gcs_uri)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,  # 设置与音频文件中相匹配的采样率
    language_code="zh-TW"
)

# 使用异步识别以处理较大的音频文件
operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout=10000)  # 设置适当的超时时间

# 转换为文本
text = " ".join([result.alternatives[0].transcript for result in response.results])
print(text)

Waiting for operation to complete...
你知道有不同的你在台灣上映的時候開放普通單半年後在中國上映票房紀錄報復嗎今天就是我們一起來了就處除三害吧後他好事執行處除三害這麼定了嗎記得今晚想吃不犯罪電影入圍的七個提名屯10月正式上映的時候在台灣的分的票房卻只有4710度報處有抄襲正一輩一部分的觀眾體質不過今年三月他的中國上有休連續使用間拿下來票房冠軍政治壓過了同期上映的好萊塢大作戰鬥門的肌肉為呢他的總票房已經突破了5.31人民幣多台灣訂第二高的路徑是1比悲傷更悲傷的故事而是中國最大的電影評分網站 日本上面的就處除三害還得老爸點1的高分子有人說跟著不定你想要撥春節檔的國產電影像是小學生寫的作業為什麼週處除三害這種口味有點重的飯的電影可以突破重重審查制度設定在中國上映和解還讓中國人艾成這個樣子呢在這之前項技能工商服務時間的時候找到對的東西嗎那就別全制霸電動螺絲起子裡面佛教六腳通通找到而且他的螺絲起子支援電動手動兩什麼是配偶參道便是讓你能夠根據需求自由切換什麼時候眼鏡手機組桌椅櫃子都能搞定在上則則預購2024旗艦組合還有優惠的千元枕上書家的手環 俄羅斯想見不用擔心弄到一定程度如果你還沒有看過什麼電影至少能在這個章節可以先斟酌觀賞他的那個新增提出處除三害這個電影的是香港導演黃金鼠在台灣的第一部作品比如何世說新語當中週處除三害的典故主要的在敘述由阮經天詩演的黑幫殺手陳桂林戶是否為自己罹患的末期肺癌決定有的生命的最後階段流明後是產出比自己排名高的兩名通緝犯因此講他的一連串的大屠殺隊到中最經典場景的就是陳桂林學習新興嶺社的螢幕他為了尋找排行第一的通緝犯了一度被靈少女感化差點受放下成見愁恨石化他發現教主題是一個騙錢站著柱子上的他槍殺了教主打歌 正常的血型和還誕生了許多黏土跟二次創作影片的光通講的我們就是那個感覺到這部電影充滿的是也是應該還有暴力與野獸的我們印像中午中國的定義審查很排斥著東西啊他到底是怎麼樣設定上面的那個進入中國的市場可不是一件容易的事如果不是兩個可怕的電影通常有兩種方式可以出口到中國燈照片跟麥片有分長片有這台灣的這邊放沒有完全的賣出版權何時跟中國的代理商合作讓他們可以發相應改變方的可以打到總票房收入的參數和解不過中國有限制有這種進口的分帳變得每年總共只能有34部分因為呢他噴的額度都沒好萊塢的大便想說台灣電影就比較能爭取到這種名的真的都選擇

In [37]:
# 将文本内容保存到文件中
with open("transcription.txt", "w") as file:
    file.write(text)

print("Transcription saved in 'transcription.txt'.")

Transcription saved in 'transcription.txt'.


In [40]:
import spacy
from spacy.lang.zh import Chinese
from spacy.lang.zh.stop_words import STOP_WORDS
from string import punctuation

In [46]:
# 加载中文模型
nlp = spacy.load("zh_core_web_sm")

# 读取文本文件
with open("transcription.txt", "r") as file:
    text = file.read()

# 使用spaCy处理文本
doc = nlp(text)

# 获取停用词列表
stopwords = list(STOP_WORDS)
punctuations = punctuation

# 文本分句和预处理
sentences = [sentence.text.strip() for sentence in doc.sents]
sentence_tokens = [sentence for sentence in doc.sents if sentence.text.strip() not in stopwords and sentence.text.strip() not in punctuations]

# 计算句子的重要性
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in stopwords or word.text.lower() in punctuations:
            continue
        if sent not in sentence_scores.keys():
            sentence_scores[sent] = 1
        else:
            sentence_scores[sent] += 1

# 基于句子数量计算摘要长度，但至少要有1个句子
summary_length = max(1, int(len(sentence_tokens) * 0.3))

# 选取分数最高的N个句子作为摘要
summary = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:summary_length]

# 显示摘要
summary_text = ' '.join([str(sentence) for sentence in summary])
print(summary_text)

裡面佛教六腳通通找到而且他的螺絲起子支援電動手動兩什麼是配偶參道便是讓你能夠根據需求自由切換什麼時候眼鏡手機組桌椅櫃子都能搞定在上則則預購2024旗艦組合還有優惠的千元枕上書家的手環 俄羅斯想見不用擔心弄到一定程度如果你還沒有看過什麼電影至少能在這個章節可以先斟酌觀賞他的那個新增提出處除三害這個電影的是香港導演黃金鼠在台灣的第一部作品比如何世說新語當中週處除三害的典故主要的在敘述由阮經天詩演的黑幫殺手陳桂林戶是否為自己罹患的末期肺癌決定有的生命的最後階段流明後是產出比自己排名高的兩名通緝犯因此講他的一連串的大屠殺隊到中最經典場景的就是陳桂林學習新興嶺社的螢幕他為了尋找排行第一的通緝犯了一度被靈少女感化差點受放下成見愁恨石化他發現教主題是一個騙錢站著柱子上的他槍殺了教主打歌 正常的血型和還誕生了許多黏土跟二次創作影片的光通講的我們就是那個感覺到這部電影充滿的是也是應該還有暴力與野獸的我們印像中午中國的定義審查很排斥著東西啊他到底是怎麼樣設定上面的那個進入中國的市場可不是一件容易的事如果不是兩個可怕的電影通常有兩種方式可以出口到中國燈照片跟麥片有分長片有這台灣的這邊放沒有完全的賣出版權何時跟中國的代理商合作讓他們可以發相應改變方的可以打到總票房收入的參數和解不過中國有限制有這種進口的分帳變得每年總共只能有34部分因為呢他噴的額度都沒好萊塢的大便想說台灣電影就比較能爭取到這種名的真的都選擇用買斷電的方式進入中國市場買斷的差別是台灣製片方會把飯吃完你出去最後 預防攝護怎樣他們都無法成本推週處除三害人走的就是買斷片方式給他去年就已經在台灣跟香港上映過也上網的袋子然後來這間公司有曾經他們其實呢是根中國代理商保理分帳指具體的數字信解綁辮頭痛的話就很好的是透過哪一種方式所有出口到中國的電影同面對就是嚴格的審查制度任何小在中國上映的電影都必須要通過中國廣播電視總局的審查審查了下雪電影審查委會負責這個我也會很神秘有沒有公佈據點名單只知道總共有36個人來自各行各業很會審查所有要上映的電影如果裡面的有他們認為的煽動抗爭危害國家安全的內容不管你是中國本土啊不是好萊塢大片對被要求刪除或修改另外中國也沒有分立制度所有定都是何 達觀賞最絕對不會出現暴力色情會宣揚邪教的內容過去能夠順利過世的台灣的通常呢只想想見你電影版這些年我們一起追的女孩子類的青春愛情片要任意外的同步的審查這部電影上面有許多血腥暴

In [50]:
print("句子数量：", len(sentences))
print("过滤后的句子数量：", len(sentence_tokens))
print("摘要长度：", summary_length)
print("文本内容：", text[:500])  # 打印部分文本内容检查
print("句子数量：", len(sentences))
print("过滤后的句子数量：", len(sentence_tokens))
print("摘要长度：", summary_length)
print("摘要内容：", summary_text)

句子数量： 3
过滤后的句子数量： 3
摘要长度： 1
文本内容： 你知道有不同的你在台灣上映的時候開放普通單半年後在中國上映票房紀錄報復嗎今天就是我們一起來了就處除三害吧後他好事執行處除三害這麼定了嗎記得今晚想吃不犯罪電影入圍的七個提名屯10月正式上映的時候在台灣的分的票房卻只有4710度報處有抄襲正一輩一部分的觀眾體質不過今年三月他的中國上有休連續使用間拿下來票房冠軍政治壓過了同期上映的好萊塢大作戰鬥門的肌肉為呢他的總票房已經突破了5.31人民幣多台灣訂第二高的路徑是1比悲傷更悲傷的故事而是中國最大的電影評分網站 日本上面的就處除三害還得老爸點1的高分子有人說跟著不定你想要撥春節檔的國產電影像是小學生寫的作業為什麼週處除三害這種口味有點重的飯的電影可以突破重重審查制度設定在中國上映和解還讓中國人艾成這個樣子呢在這之前項技能工商服務時間的時候找到對的東西嗎那就別全制霸電動螺絲起子裡面佛教六腳通通找到而且他的螺絲起子支援電動手動兩什麼是配偶參道便是讓你能夠根據需求自由切換什麼時候眼鏡手機組桌椅櫃子都能搞定在上則則預購2024旗艦組合還有優惠的千元枕上書家的手環 俄羅斯想見不用擔心弄到一定程度如果你還沒有看過什麼電影至少能在這個章節可以先斟酌觀賞他的那個
句子数量： 3
过滤后的句子数量： 3
摘要长度： 1
摘要内容： 裡面佛教六腳通通找到而且他的螺絲起子支援電動手動兩什麼是配偶參道便是讓你能夠根據需求自由切換什麼時候眼鏡手機組桌椅櫃子都能搞定在上則則預購2024旗艦組合還有優惠的千元枕上書家的手環 俄羅斯想見不用擔心弄到一定程度如果你還沒有看過什麼電影至少能在這個章節可以先斟酌觀賞他的那個新增提出處除三害這個電影的是香港導演黃金鼠在台灣的第一部作品比如何世說新語當中週處除三害的典故主要的在敘述由阮經天詩演的黑幫殺手陳桂林戶是否為自己罹患的末期肺癌決定有的生命的最後階段流明後是產出比自己排名高的兩名通緝犯因此講他的一連串的大屠殺隊到中最經典場景的就是陳桂林學習新興嶺社的螢幕他為了尋找排行第一的通緝犯了一度被靈少女感化差點受放下成見愁恨石化他發現教主題是一個騙錢站著柱子上的他槍殺了教主打歌 正常的血型和還誕生了許多黏土跟二次創作影片的光通講的我們就是那個感覺到這部電影充滿的是也是應該還有暴力與野獸的我們印像中午中國的定義審查很排斥著東西啊他到底是怎麼樣設定上面的那個進

In [64]:
import jieba
from gensim.models import Word2Vec
from collections import Counter

# 读取文本文件
with open("transcription.txt", "r", encoding='utf-8') as file:
    text = file.read()

# 加载停用词
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f.readlines()]

# 向jieba字典中添加特定关键词
jieba.add_word('週處除三害')

# 使用jieba进行中文分词
words = jieba.cut(text)

# 过滤长度小于2的词和停用词
filtered_words = [word for word in words if len(word) > 1 and word not in stopwords]

# 计算词频
word_counts = Counter(filtered_words)

# 获取最常出现的10个词
top_ten = word_counts.most_common(10)

# 打印结果
print("最常出現的10個關鍵字及其出現次數：")
for word, count in top_ten:
    print(f"{word}: {count}")

# Word2Vec分析
sentences = [filtered_words]  # Word2Vec模型训练需要句子列表作为输入
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 获取与特定词语相似的词语示例
similar_words = model.wv.most_similar(filtered_words[0], topn=10)
print(f"\與‘{filtered_words[0]}’最相似的10個詞及相似度：")
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

最常出现的10个关键字及其出现次数：
中國: 32
電影: 19
台灣: 9
上映: 9
審查: 8
週處除三害: 7
票房: 6
宗教: 5
觀眾: 4
制度: 4

与‘不同’最相似的10个词及相似度：
符合: 0.3532281219959259
分長: 0.3161581754684448
部分: 0.31496211886405945
出國: 0.29390719532966614
使用: 0.28494688868522644
這樣子: 0.28434762358665466
相定: 0.2644304931163788
順利過: 0.24681925773620605
擔心: 0.23865383863449097
今晚: 0.23311688005924225
